In [1]:
using Oscar

$\require{action}$

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2020 by The Oscar Development Team


# Groups in Oscar

Oscar handles group theoretic questions via combining different systems: GAP, Hecke, AbstractAlgebra, ...

Growing documentation can be found at the following link: 

https://oscar-system.github.io/Oscar.jl/previews/PR209/Groups/groups.html#Base.one-Tuple{Oscar.GAPGroup}

Let's see how to explore Rosa's group using Oscar.

# Rosa's group
Rosa gave us a group in terms of its ID in **GAP's SmallGroups library**. What can we say about this group? 

In [2]:
G = small_group(8,3)

<pc group of size 8 with 3 generators>

The group's ID already tells us that $G$ has 8 elements. Moreover, we know that $G$ can be generated by 3 elements (not the minimum number of generators necessarily!).

The number of (isomorphism classes of) groups of order $8=2^3$ is:

In [3]:
number_small_groups(8)

5

A **curiosity**! The number of groups of order $1024=2^{10}$ is: 

In [4]:
number_small_groups(1024)

49487365422

which is more than 99% of all groups of order at most 2000!

So going back to our investigation, how can we determine what group $G$ is?

In [5]:
isabelian(G)

false

So we know that $G$ is not abelian, which leaves us with only 2 possibilities.. $D_8$ and $Q_8$. Indeed: 

In [6]:
p=2
for i in 1:5
    T = small_group(p^3,i);
    K = derived_subgroup(T)[1];
    println(K)
    println(isabelian(T))
end

Group([  ])
true
Group([  ])
true
Group([ f3 ])
false
Group([ f3 ])
false
Group([  ])
true


and we also know from the Elementary Divisors Theorem that every abelian group of order 8 is isomorphic to one of: $\mathbb{Z}/(8)$, $\mathbb{Z}/(4)\times\mathbb{Z}/(2)$, or $\mathbb{Z}/(2)\times\mathbb{Z}/(2)\times \mathbb{Z}/(2)$. 

Another way:

In [7]:
p=2
for i in 1:5
    T = small_group(p^3,i);
    if isabelian(T)==false
        println(i)
    end
end

3
4


Define now:

In [8]:
D = dihedral_group(8)
#small_group_identification(D)

<pc group of size 8 with 3 generators>

In [9]:
Q = quaternion_group(8)
#small_group_identification(Q)

<pc group of size 8 with 3 generators>

# (Symmetric and) Permutation groups

Recall that the group $G$ was born as the Galois group associated to the polynomial

$$
f(x)=x^8 - 21x^6 + 84x^4 - 105x^2 + 25
$$

and thus naturally as a subgroup of $\mathrm{S}_8$. 

An oracle (or Rosa?) tells me that there is a labeling of the roots $\alpha_1,\ldots, \alpha_8$ of $f$ such that the group $G$ is generated by the following elements of $\mathrm{S}_8$:

$$
(1,8)(2,7)(3,6)(4,5), \quad (1,2)(3,5)(4,6)(7,8), \quad (1,5,8,4)(2,6,7,3).
$$

Let's check these guys generate a copy of $D_8$ in $\mathrm{S}_8$!

To define a symmetric group:

In [10]:
S = symmetric_group(8)

Sym( [ 1 .. 8 ] )

How to define a cycle in the symmetric group:

In [11]:
cperm([1,2,3,4])

(1,2,3,4)

We now define the generators of our Galois group as a subgroup of the symmetric group of degree 8:

In [12]:
s=cperm([1,8])*cperm([2,7])*cperm([3,6])*cperm([4,5])

(1,8)(2,7)(3,6)(4,5)

In [13]:
t=cperm([1,2])*cperm([3,5])*cperm([4,6])*cperm([7,8])

(1,2)(3,5)(4,6)(7,8)

In [14]:
u=cperm([1,5,8,4])*cperm([2,6,7,3])

(1,5,8,4)(2,6,7,3)

Defining $G$ via its generators as a subgroup of $\mathrm{S}_8$ returns a pair consisting of the group $G$ together with an embedding $G\rightarrow \mathrm{S}_8$. If you want to just work with the group, don't forget to **just take** the first entry!

In [15]:
H = sub(S,[s,t,u])#[1] #Here G is called H!

(Group([ (1,8)(2,7)(3,6)(4,5), (1,2)(3,5)(4,6)(7,8), (1,5,8,4)(2,6,7,3) ]), Group homomorphism from 
Group([ (1,8)(2,7)(3,6)(4,5), (1,2)(3,5)(4,6)(7,8), (1,5,8,4)(2,6,7,3) ])
to
Sym( [ 1 .. 8 ] )
)

In [23]:
small_group_identification(H[1])

(8, 3)

# Types and methods

There is a number of other questions one can ask about the group $G$: some *methods are only implemented for specific types*! The types available in Oscar are:

* groups of permutations 
* groups of matrices
* finitely presented groups
* polycyclic groups
* direct products of two groups
* automorphism groups

In [17]:
typeof(H)

Tuple{PermGroup,Oscar.GAPGroupHomomorphism{PermGroup,PermGroup}}

In [18]:
typeof(G)

PcGroup

In [19]:
methodswith(typeof(H))
#istransitive(H)
#number_moved_points(H)

[]

In [20]:
#istransitive(G) #returns an error!

Some GAP commands for computing normal series in G are also available:

In [21]:
derived_series(G)

3-element Array{PcGroup,1}:
 <pc group of size 8 with 3 generators>
 Group([ f3 ])
 Group([  ])

And even if the GAP command has not been translated into Oscar, you can always call for GAP directly: 

In [22]:
GAP.Globals.LowerCentralSeries(GAP.Globals.SmallGroup(8,3))

GAP: [ <pc group of size 8 with 3 generators>, Group([ f3 ]), Group([ <identity> of ... ]) ]